# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2024-03-31 10:36:51 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [2]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  558.512 ms (2 allocations: 32.00 MiB)


In [3]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

No GPU found.


In [12]:
using Pkg
Pkg.add("Graphs")
Pkg.add("GraphIO")
#Pkg.add("Plots")
#Pkg.add("LightGraphs")
Pkg.add("Random")
Pkg.add("StatsBase")
Pkg.add("Distributions")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Small-world model

In [29]:
using Graphs
using GraphIO

function small_world_network(n::Int, k::Int, p::Float64)

  g = Graphs.SimpleGraph(n)
  neighbor=div(k,2)

  #We create a ring network
  for node in 1:n
    for k in 1:neighbor
      k1=node+k
      k2=node+n-k

      if k1>n
        Graphs.add_edge!(g, node, k1%n)
      else
        Graphs.add_edge!(g, node, k1)
      end

      if k2>n
        Graphs.add_edge!(g, node, k2%n)
      else
        Graphs.add_edge!(g, node, k2)
      end
    end
  end

  #We change randomly edge connections to add higher clustering
  for edge in Graphs.edges(g)
    if rand() < p

      u, v = Graphs.src(edge), Graphs.dst(edge)
      target = rand(1:n)

      #Not self loops or previous or multi-edge allowed
      while target == u || Graphs.has_edge(g, u, target)
          target = rand(1:n)
      end

      Graphs.rem_edge!(g, edge)
      Graphs.add_edge!(g, u, target)

    end
  end

  return g

end

small_world_network (generic function with 1 method)

In [21]:
using Graphs

Graphs.SimpleGraph(3)

{3, 0} undirected simple Int64 graph

Saving graph in NET format

In [36]:
using Graphs

 # Save the network to a Pajek (.net) file
function savegraph_net(g::Graphs.SimpleGraph{Int}, gname::String)

  num_vertices=Graphs.nv(g)
  file=gname*".net"

  open(file, "w") do io
      println(io, "*Vertices ", num_vertices)

      for v in 1:num_vertices
        println(io, v, ' ','"', v, '"')
      end

      println(io, "*Edges")
      for e in Graphs.edges(g)
          println(io, Graphs.src(e), ' ', Graphs.dst(e))
      end
  end
end

savegraph_net (generic function with 1 method)

In [ ]:
savegraph_net(g,"Test3")

Test using the Julia functions

In [15]:
using Graphs
test=Graphs.newman_watts_strogatz(1000, 15, 0.3)


{1000, 9114} undirected simple Int64 graph

In [39]:
test2=small_world_network(1000, 15, 1.0)

{1000, 7000} undirected simple Int64 graph

In [40]:
savegraph_net(test2,"ok")

In [ ]:
degree(test2)

1000-element Vector{Int64}:
 16
 17
 15
 15
 15
 15
 17
 18
 17
 14
 16
 13
 15
  ⋮
 11
 14
 13
 14
 13
 13
 16
 12
 12
 13
 15
 10

Configuration Model

In [80]:
using Graphs
using Random

function configuration_model(degree_distribution::Vector{Int})

    num_nodes = length(degree_distribution)


    total_stubs = sum(degree_distribution)

    # In case total degrees is odd, make it even
    if total_stubs % 2 != 0
        idx = rand(1:num_nodes)
        degree_distribution[idx] += 1
        total_stubs += 1
    end

    # Create as many stubs as the degree of each node
    stubs = []
    for (node, degree) in enumerate(degree_distribution)
        for _ in 1:degree
            push!(stubs, node)
        end
    end

    # Randomly order the stubs
    Random.shuffle!(stubs)

    graph = Graphs.SimpleGraph(num_nodes)

    while length(stubs) >= 2
        u = pop!(stubs)
        v = pop!(stubs)

        # Avoiding self-loops and multiple edges
        if u != v && !Graphs.has_edge(graph, u, v)
            Graphs.add_edge!(graph, u, v)
        end
    end

    return graph
end

configuration_model (generic function with 2 methods)

In [81]:
degree_distribution = [5, 3, 2, 4, 6, 1, 3]
network = configuration_model(degree_distribution)

{7, 8} undirected simple Int64 graph

In [82]:
savegraph_net(network,"CM")

In [83]:
Graphs.degree(network)

7-element Vector{Int64}:
 4
 2
 2
 2
 3
 1
 2

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />